In [1]:
## packages import from run

from functools import partial, update_wrapper
from threading import Event, Lock
from collections import deque
from dataclasses import dataclass
from datetime import datetime
from random import shuffle, sample
from pathlib import Path
from typing import Dict, Any, Optional
from dataclasses import asdict
import hashlib
import logging
import argparse
import shutil
import json
import sys

import ase
from ase.db import connect
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from colmena.models import Result
from colmena.queue import ColmenaQueues
from colmena.queue.redis import RedisQueues
from colmena.thinker import BaseThinker, event_responder, result_processor, ResourceCounter, task_submitter
import proxystore as ps
import numpy as np
import torch
from proxystore.store import register_store
from proxystore.store.file import FileStore
from proxystore.store.globus import GlobusStore, GlobusEndpoints
from proxystore.store.redis import RedisStore
from proxystore.store.utils import get_key

from fff.learning.gc.ase import SchnetCalculator
from fff.learning.gc.functions import GCSchNetForcefield
from fff.learning.gc.models import SchNet, load_pretrained_model
from fff.learning.util.messages import TorchMessage
from fff.sampling.md import MolecularDynamics
from fff.simulation import run_calculator
from fff.simulation.utils import read_from_string, write_to_string
from _pytest.fixtures import fixture
from ase.build import molecule

from config import wsl as make_config

In [2]:
## packages for monitor cpu and gpu
import psutil
import pynvml
import time
import os
from memory_profiler import profile

In [ ]:
## test finetune-surrogate model train and infernece


In [ ]:
## test sampling task

In [15]:
## test simulation task
calc = dict(calc='psi4', method='pbe0-d3', basis='aug-cc-pvdz', num_threads=64)
temp_path = "./temp"
## print complete whole path
path = os.path.abspath(temp_path)
print(path)

def _wrap(func, **kwargs):
    out = partial(func, **kwargs)
    update_wrapper(out, func)
    return out

# @fixture()
def atoms():
    return molecule('H2O')
atom = atoms()

# @fixture()
def cluster():
    xyz = """30

O       7.581982610000000     -0.663324770000000      5.483883860000000
H       8.362350460000000     -0.079370470000000      5.498567580000000
H       7.846055030000000     -1.464757200000000      5.041030880000000
O       9.456702229999999      1.642301080000000      8.570644379999999
H      10.114471399999999      1.655581000000000      9.261547090000001
H       9.181962009999999      2.562770840000000      8.428308489999999
O       9.664885520000000      1.027763610000000      5.758778100000000
H       9.485557560000000      1.914335850000000      5.411871910000000
H       9.760457990000001      1.144007330000000      6.710969450000000
O       6.000383380000000      4.009448050000000      7.349214080000000
H       5.983903880000000      4.025474550000000      6.383275510000000
H       5.536083220000000      3.203337670000000      7.608772750000000
O       4.833731170000000      1.482195020000000      7.883007530000000
H       5.628127100000000      0.955450120000000      8.084721569999999
H       4.134047510000000      1.149705890000000      8.438218120000000
O       7.110025880000000      0.051394890000000      8.205573080000001
H       7.262372970000000     -0.325556960000000      7.328944680000000
H       7.906465050000000      0.552607360000000      8.419908520000000
O       6.173881530000000      3.688445090000000      4.528872010000000
H       5.701079370000000      4.022632120000000      3.771772860000000
H       5.903837200000000      2.759263990000000      4.641063690000000
O       5.429551600000000      1.145089270000000      5.097751140000000
H       6.135000710000000      0.486118580000000      5.133624550000000
H       5.085167410000000      1.211727260000000      5.997292520000000
O       8.597597120000000      4.222480770000000      8.031750680000000
H       7.641802790000000      4.166800020000000      7.848542690000000
H       8.760176660000001      5.097825050000000      8.372748370000000
O       8.954336169999999      3.647526740000000      5.177083970000000
H       8.927373890000000      3.954237700000000      6.090421680000000
H       8.043264389999999      3.698852060000000      4.860042570000000
"""
    return read_from_string(xyz, 'xyz')
xyz = cluster()
xyz = write_to_string(xyz, 'xyz')
print(xyz)
xyz = write_to_string(atom, 'xyz')
print(xyz)
my_run_simulation = _wrap(run_calculator, calc=calc, temp_path=path)


/home/yxx/work/project/colmena/multisite_/finetuning-surrogates/runs/analysis/temp
30

O       7.581982610000000     -0.663324770000000      5.483883860000000
H       8.362350460000000     -0.079370470000000      5.498567580000000
H       7.846055030000000     -1.464757200000000      5.041030880000000
O       9.456702229999999      1.642301080000000      8.570644379999999
H      10.114471399999999      1.655581000000000      9.261547090000001
H       9.181962009999999      2.562770840000000      8.428308489999999
O       9.664885520000000      1.027763610000000      5.758778100000000
H       9.485557560000000      1.914335850000000      5.411871910000000
H       9.760457990000001      1.144007330000000      6.710969450000000
O       6.000383380000000      4.009448050000000      7.349214080000000
H       5.983903880000000      4.025474550000000      6.383275510000000
H       5.536083220000000      3.203337670000000      7.608772750000000
O       4.833731170000000      1.482195020000000 

In [6]:
%%memit
res = my_run_simulation(xyz)

UsageError: Cell magic `%%memit` not found.


In [7]:
## gpu monitor test
# 初始化pynvml
pynvml.nvmlInit()

# 获取GPU数量
num_gpus = pynvml.nvmlDeviceGetCount()

# 遍历每个GPU
for i in range(num_gpus):
    # 获取GPU句柄
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)

    # 获取GPU的利用率
    utilization = pynvml.nvmlDeviceGetUtilizationRates(handle).gpu

    # 获取GPU的温度
    temperature = pynvml.nvmlDeviceGetTemperature(handle, pynvml.NVML_TEMPERATURE_GPU)
    
    # 获取GPU的内存信息
    memory_info = pynvml.nvmlDeviceGetMemoryInfo(handle)

    # 打印GPU内存信息
    print(f"GPU {i+1}:")
    print(f"Total Memory: {memory_info.total / (1024**2)} MB")
    print(f"Used Memory: {memory_info.used / (1024**2)} MB")
    print(f"Free Memory: {memory_info.free / (1024**2)} MB")

    # 打印GPU信息
    print(f"GPU {i+1}: Utilization = {utilization}%, Temperature = {temperature}°C")

# 清理pynvml
pynvml.nvmlShutdown()

GPU 1:
Total Memory: 6144.0 MB
Used Memory: 3831.078125 MB
Free Memory: 2312.921875 MB
GPU 1: Utilization = 1%, Temperature = 56°C


In [49]:
## cpu monitor test
# 获取CPU利用率
cpu_percent = psutil.cpu_percent(interval=1)
print(f"CPU Utilization: {cpu_percent}%")

# 获取内存使用情况
memory = psutil.virtual_memory()
print(f"Total Memory: {memory.total / (1024**3):.2f} GB")
print(f"Available Memory: {memory.available / (1024**3):.2f} GB")
print(f"Used Memory: {memory.used / (1024**3):.2f} GB")

# 获取磁盘I/O信息
disk_io = psutil.disk_io_counters()
print(f"Disk Read Count: {disk_io.read_count}")
print(f"Disk Write Count: {disk_io.write_count}")
print(f"Disk Read Bytes: {disk_io.read_bytes / (1024**2):.2f} MB")
print(f"Disk Write Bytes: {disk_io.write_bytes / (1024**2):.2f} MB")

# 等待一段时间
# time.sleep(1)
import os
# 定义测试文件路径和大小
test_file = 'disk_io_test_file.bin'
file_size = 1024 * 1024 * 100  # 100 MB
# 创建测试文件
with open(test_file, 'wb') as f:
    f.seek(file_size - 1)
    f.write(b'\0')

# 读取测试文件
# 获取初始的磁盘I/O信息
disk_io_start = psutil.disk_io_counters()
with open(test_file, 'rb') as f:
    data = f.read()
print(f"File size: {file_size / (1024**2)} MB")
print("Disk I/O test completed.")

# 删除测试文件
# os.remove(test_file)

# 获取最终的磁盘I/O信息
disk_io_end = psutil.disk_io_counters()

# 计算磁盘I/O增量数据
disk_io_diff = {}
disk_io_diff['read_count'] = disk_io_end.read_count - disk_io_start.read_count
disk_io_diff['write_count'] = disk_io_end.write_count - disk_io_start.write_count
disk_io_diff['read_bytes'] = disk_io_end.read_bytes - disk_io_start.read_bytes
disk_io_diff['write_bytes'] = disk_io_end.write_bytes - disk_io_start.write_bytes

# 打印增量数据
print(f"Disk Read Count: {disk_io_diff['read_count']}")
print(f"Disk Write Count: {disk_io_diff['write_count']}")
print(f"Disk Read Bytes: {disk_io_diff['read_bytes'] / (1024**2):.2f} MB")
print(f"Disk Write Bytes: {disk_io_diff['write_bytes'] / (1024**2):.2f} MB")

CPU Utilization: 0.5%
Total Memory: 15.62 GB
Available Memory: 4.17 GB
Used Memory: 11.12 GB
Disk Read Count: 421657
Disk Write Count: 1742382
Disk Read Bytes: 6572.50 MB
Disk Write Bytes: 26248.35 MB
File size: 100.0 MB
Disk I/O test completed.
Disk Read Count: 0
Disk Write Count: 1
Disk Read Bytes: 0.00 MB
Disk Write Bytes: 0.00 MB


In [47]:
import os
import psutil

# 定义测试文件路径和大小
test_file = 'disk_io_test_file.bin'
file_size = 1024 * 1024 * 1000 # 100 MB

# 创建测试文件
with open(test_file, 'wb') as f:
    f.seek(file_size - 1)
    f.write(b'\0')

# 获取初始的磁盘I/O信息
disk_io_start = psutil.disk_io_counters()

# 读取测试文件
with open(test_file, 'rb') as f:
    data = f.read()

# 获取最终的磁盘I/O信息
disk_io_end = psutil.disk_io_counters()

# 计算磁盘I/O增量数据
# disk_io_diff = disk_io_end - disk_io_start

# 打印增量数据
print(f"Disk Read Bytes: {(disk_io_end.read_bytes - disk_io_start.read_bytes) / (1024**2)} MB")
print(f"Disk Write Bytes: {(disk_io_end.write_bytes - disk_io_start.write_bytes) / (1024**2)} MB")

print("Disk I/O test completed.")

# 删除测试文件
os.remove(test_file)

Disk Read Bytes: 0.0 MB
Disk Write Bytes: 0.109375 MB
Disk I/O test completed.
